# Задания

Курсовой проект состоит из 3х частей
Студенты весь курс работают с retail-датасетом:  
данные о покупках 2.5k клиентов 80k товаров, признаковое описание пользователей и товаров.  
3 недели покупок недоступны студентам. По мере развития курса мы строим все более сложные и продвинутые RS, которые улучшают качество рекомендаций.  
В конце курса студентам предлагается построить алгоритм на исторических данных и предсказать какие товары пользователи купят в будущем.  
Для обучения доступны 95 недель + информация о пользователях и товарах  
96 - 98 недели недоступны. Нужно предсказать какие товары пользователи купят за эти 3 недели  
Качество оценивается по метрике precision@5, усредненной по всем пользователям. Критерий сдачи: значение метрики > 0.27  
Во время разработки модели выделите 3 известные недели (93 - 95) в качестве валидационной выборки.  
Продемонстрируйте на валидационной выборке требуемое значение precision@5  
Ноутбук должен содержать описание pipeline’а Вашей модели. (Какие модели использовали, какие признаки генерировали и тд)

Бизнес-ограничения:
- Нельзя рекомендовать top 3 самых популярных товаров
- Нельзя рекомендовать товары, которые не продавались последние 12 месяцев
- Нельзя рекомендовать товары, с общим числом продаж < 50
- Нельзя рекомендовать товары, которые стоят < 1$
- Каждому пользователю нужно порекомендовать 5 товаров, один из них должен быть обязательно товаром, который данный пользователь никогда не покупал



В рамках решения основной задачи студенты должны произвести предварительный и визуальный анализ данных. В рамках его нужно построить диаграммы распределения признаков, проанализировать некоторые категории товаров, их цену, среднее число покупок и тд. Это творческое задание, единственное конкретное требование: каждый студент должен найти в данных что-то уникальное (то чего не нашли одногруппники).
	

* Провести некоторое исследование по любой теме, разобранной в курсе, тему исследования необходимо согласовать с преподавателем.


# Решение

### Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.main_recommender import MainRecommender

In [2]:
data = pd.read_csv('../data/retail_train.csv')
item_features = pd.read_csv('../data/product.csv')
user_features = pd.read_csv('../data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

**Нельзя рекомендовать товары, которые не продавались последние 12 месяцев**

In [3]:
# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(10)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
5,2375,26984851516,1,826249,2,1.98,364,-0.60,1642,1,0.0,0.0
6,2375,26984851516,1,1043142,1,1.57,364,-0.68,1642,1,0.0,0.0
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0
8,2375,26984851516,1,1102651,1,1.89,364,0.00,1642,1,0.0,0.0
9,2375,26984851516,1,6423775,1,2.00,364,-0.79,1642,1,0.0,0.0


In [4]:
n_items_before = data_train_lvl_1['item_id'].nunique()
n_items_before

83685

In [5]:
top_item_quantity = 5000
data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, top_item_quantity=top_item_quantity)

In [ ]:
  data_train_lvl_1 = data_train_lvl_1[data_train_lvl_1['quantity'] > 50]

In [6]:
n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 3703


In [5]:
recommender = MainRecommender(data_train_lvl_1)

100%|██████████| 2381/2381 [00:00<00:00, 54146.02it/s]


[1, 2, 3]